<left>Correlation One DS4A/W </left>
<br>
<h3>Team - 3</h3>
<h3>TA - Oretha Domfeh</h3>

<h2><center> Data Preprocessing </center></h2>

<center> Amber Lee </center>
<center> Cindy Zhang </center>
<center> Hejia Zhang </center>
<center> Wafer Hsu </center>
<br>

<center>Monday, Jul 24</center>


In [1]:
import sys
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install newspaper3k
!{sys.executable} -m pip install git+https://github.com/ranahaani/GNews.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 2.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=c0edc9ad0364606115b836e13f899baa201d1ab82a8fcf995bbad4c67631f720
  Stored in directory: /Users/cindy/Library/Caches/pip/wheels/94/ad/df/a2a01300cea47d5695f242f7e925a805970106fd9e4b151468
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=cf03bf4817bc84e648544148198e2ef4dc7a066d6021fb7db3451df0e4ba431b
  Stored in directory: /Users/cindy/Library/Caches/pip/wheels/43/4a/c2/61a371b2524ac90805391c660d8dc4505705297f25e2b85a5d
  Cr

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from gnews import GNews
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/cindy/nltk_data...


True

## Helper Functions

In [2]:
#Quandl Python Tables API Function
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(os.getcwd(), "quandl_data_table_downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink

    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.makedirs(table_dir)

    if date_override is None:
        my_date = dt.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    api_key = os.environ.get('Cindy_Quandl_API')
    dl = quandl.export_table(
        table_path, filename=data_file, api_key=api_key, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"

def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

## 1. Retrieving Stock Data

### 1.1 Set up

#### Universe
- S&P 500 stocks that are updated recently
- At least 5 years of price data (2018-01-01 to 2023-05-31), IPO after 2018-01-01 are excluded
- Have at least 100 trading days of price data before 2018-01-01 for rolling window Fama-French exposure calculation (IPO before 2017-08-16)
- Because of the lacking data of Fama-French 5 factors returns, the universe is updated to 2018-01-01 to 2023-04-28

In [ ]:
# Full market stock data
 full_price = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=False)
# S&P 500 tickers list
SP500 = pd.read_excel("S&P 500 tickers.xlsx")
# Fama-French 5 factors returns
F_F_5_Factors = pd.read_csv("F_F_Research_Data_5_Factors_daily.csv", skiprows = 3)
F_F_5_Factors.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
F_F_5_Factors['Date'] = pd.to_datetime(F_F_5_Factors['Date'], format='%Y%m%d')
F_F_5_Factors.head()

In [ ]:
# Only keep the stocks that are in the S&P 500
universe = full_price[full_price['ticker'].isin(SP500['Symbol'])].copy().sort_values(by = ['ticker', 'date'])
universe['date'] = pd.to_datetime(universe['date'])
# Only keep the stocks that are in the S&P 500
universe = full_price[full_price['ticker'].isin(SP500['Symbol'])].copy().sort_values(by = ['ticker', 'date'])
universe['date'] = pd.to_datetime(universe['date'])
universe.drop(universe[(universe.ticker.isin(SP500_start_date[SP500_start_date.start_date > '2017-08-16'].ticker))].index, inplace = True)

In [ ]:
# Set up the universe time series precisely
universe = universe[(universe['date'] >= '2017-08-15') & (universe['date'] < '2023-06-01')].sort_values(by = ['ticker', 'date'])[['ticker', 'date', 'adj_close', 'adj_volume']].reset_index(drop = True)
# Calculate the daily return for each ticker
universe['return'] = universe.groupby('ticker')['adj_close'].pct_change()
universe = universe.dropna()

### 1.2 F-F 5 factors

- The Fama-French 5 factors returns are updated daily.
- The Fama-French 5 factors returns are risk premiums of the market which are not directly comparable to the returns of the stocks.
- We will run the regression of daily excess returns of each stock on the Fama-French 5 factors to get the beta values as that stock's exposures to each risk premium in the model.
- To prevent look ahead bias, I will use the expected return of the previous 100 days as the exposure to each risk premium of th next day. In other words, we will run 100 trading days rolling window regression of daily excess returns against Fama-French 5 factors returns.
- Since the most recent factor return data that is available is 2023-04-28, our universe is updated to 2018-01-01 to 2023-04-28.

In [ ]:
# Match the date of Fama-French 5 factors to the universe date time series
F_F_5_Factors = F_F_5_Factors[(F_F_5_Factors['Date'] >= '2017-08-15') & (F_F_5_Factors['Date'] < '2023-06-01')].reset_index(drop = True)
F_F_regression = universe.merge(F_F_5_Factors, left_on = 'date', right_on = 'Date', how = 'left')
F_F_regression['excess_return'] = F_F_regression['return'] - F_F_regression['RF']
# The time range of the universe shrinks because of the lacking information of Fama-French 5 factor returns data
F_F_regression = F_F_regression.dropna()

In [ ]:
# Run 100 trading days rolling window regression of daily excess returns against Fama-French 5 factors returns
F_F_regression['Mkt-RF_beta'] = np.nan
F_F_regression['SMB_beta'] = np.nan
F_F_regression['HML_beta'] = np.nan
F_F_regression['RMW_beta'] = np.nan
F_F_regression['CMA_beta'] = np.nan
for ticker, group in F_F_regression.groupby('ticker'):
    for idx in range(100, len(group)):
        X = group.loc[group.index[idx] - 100:group.index[idx] - 1, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
        y = group.loc[group.index[idx] - 100:group.index[idx] - 1, 'excess_return']
        #print(group.loc[group.index[idx]])
        coefficients = np.linalg.inv(X.T @ X) @ X.T @ y
        F_F_regression.loc[group.index[idx], 'Mkt-RF_beta'] = coefficients[0]
        F_F_regression.loc[group.index[idx], 'SMB_beta'] = coefficients[1]
        F_F_regression.loc[group.index[idx], 'HML_beta'] = coefficients[2]
        F_F_regression.loc[group.index[idx], 'RMW_beta'] = coefficients[3]
        F_F_regression.loc[group.index[idx], 'CMA_beta'] = coefficients[4]
        #print(group.loc[group.index[idx]])

In [ ]:
# Drop the unnecessary columns which are market risk premiums
F_F_regression.drop(columns=['Date','Mkt-RF','SMB','HML','RMW','CMA','RF','excess_return'], inplace=True)
F_F_regression = F_F_regression.dropna()
F_F_regression.to_pickle('stock_universe.pkl')

In [3]:
universe_check = pd.read_pickle('stock_universe.pkl')
universe_check

,ticker,date,adj_close,adj_volume,return,Mkt-RF_beta,SMB_beta,HML_beta,RMW_beta,CMA_beta
100,A,2018-01-09,69.013902,2666711.0,0.024554,0.009027,0.001063,-0.007793,-0.004662,-0.005455
101,A,2018-01-10,68.071536,2957184.0,-0.013655,0.008722,0.000859,-0.006795,-0.006242,-0.005700
102,A,2018-01-11,68.081152,1511134.0,0.000141,0.007732,0.001027,-0.007521,-0.005757,-0.005344
103,A,2018-01-12,68.975438,1448155.0,0.013136,0.007377,0.000628,-0.007505,-0.005661,-0.005886
104,A,2018-01-16,68.494639,1703398.0,-0.006971,0.007580,0.000491,-0.007457,-0.005694,-0.005725
...,...,...,...,...,...,...,...,...,...,...
706757,ZTS,2023-04-24,176.350000,1117741.0,-0.002996,0.008080,0.004188,0.000523,-0.001081,-0.001064
706758,ZTS,2023-04-25,173.020000,1151148.0,-0.018883,0.007815,0.004184,0.000368,-0.001543,-0.000980
706759,ZTS,2023-04-26,172.940000,1205198.0,-0.000462,0.007942,0.004660,0.000837,-0.001425,-0.001464
706760,ZTS,2023-04-27,173.950000,1682893.0,0.005840,0.008008,0.004670,0.001035,-0.001673,-0.001201


## 2. Retrieving Headlines

### 2.1 Set up

You may need to install the libraries `beautifulsoup4` and `newspaper3k`.

The `GNews` library needs to be installed  from the Github source. Here is a [StackOverflow forum] I referenced, in case it is helpful.

After discussion, we plan to retrieve 5 equities for this project and they are: `AAPL`, `AMZN`, `FB`, `GOOG`, `MSFT`. Thus, the news data will only be retrieved for these 5 equities.

### 2.2 Function

In [3]:
# set up the 2 week periods

first_day = np.ones(12, dtype = int)
middle_day = np.repeat(15, 12)
middle_day[1] = 14 # feb
last_day = np.tile([31, 30], 6)
last_day[7:12] = last_day[0:5]
last_day[1] = 28 # feb

start_days = []
end_days = []

for i in range(12):
    
    start_days.append(first_day[i])
    end_days.append(middle_day[i])
    
    start_days.append(middle_day[i])
    end_days.append(last_day[i])

months = np.repeat(range(12), 2) + 1

# print(start_days)
# print(end_days)
# print(months)

In [4]:
def get_headlines(year, keyword):
    """
    year: int 
    keyword: str, the company name
    """
    
    headlines_df = pd.DataFrame(columns = ["date", "title", "publisher"])
    
    for two_week_period in range(24):
    
        month = months[two_week_period]
        start_day = start_days[two_week_period]
        end_day = end_days[two_week_period]

        start = dt.datetime(year, month, start_day)
        end = dt.datetime(year, month, end_day)

        gnews = GNews(language = "en",
                      start_date = start, 
                      end_date = end)

        news_df = pd.DataFrame(gnews.get_news(keyword))

        if news_df.shape == (0, 0):
            print(f"No news between {start} and {end} for {keyword}.\n")
            continue

        news_df['date'] = pd.to_datetime(news_df['published date'])

        headlines_df = pd.concat([headlines_df, news_df[['date', 'title', 'publisher']].copy()],
                                 ignore_index = True)
    
    return headlines_df

### 2.3 Retrieve data

Only run one cell at a time!

When running these cells, you will get errors about having no news for certain time periods. That's fine, don't re-run the cell. Keep it the way it is so we have records about when the headlines were missing. Just commit and push what you have from that one run.

One day later, it can be helpful to duplicate the cell, change the `range(2018, 2023+1)` to start from whichever year there is missing headlines, and run the code again.

In [ ]:
# wafer: apple, amazon

In [ ]:
# apple

company = "Apple"

for year in range(2018, 2023+1):

    headlines_df = get_headlines(year, company)

    file = "headlines/" + str(year) + "_" + company + "_headlines.csv"
    headlines_df.to_csv(file, index = False)

In [ ]:
# amazon

company = "Amazon"

for year in range(2018, 2023+1):

    headlines_df = get_headlines(year, company)

    file = "headlines/" + str(year) + "_" + company + "_headlines.csv"
    headlines_df.to_csv(file, index = False)

In [ ]:
# cindy: nvidia, microsoft

In [78]:
# nvidia

company = "Google"

for year in range(2018, 2023+1):

    headlines_df = get_headlines(year, company)

    file = "headlines/" + str(year) + "_" + company + "_headlines.csv"
    headlines_df.to_csv(file, index = False)

No news between 2023-07-15 00:00:00 and 2023-07-31 00:00:00 for Google.

No news between 2023-08-01 00:00:00 and 2023-08-15 00:00:00 for Google.

No news between 2023-08-15 00:00:00 and 2023-08-31 00:00:00 for Google.

No news between 2023-09-01 00:00:00 and 2023-09-15 00:00:00 for Google.

No news between 2023-09-15 00:00:00 and 2023-09-30 00:00:00 for Google.

No news between 2023-10-01 00:00:00 and 2023-10-15 00:00:00 for Google.

No news between 2023-10-15 00:00:00 and 2023-10-31 00:00:00 for Google.

No news between 2023-11-01 00:00:00 and 2023-11-15 00:00:00 for Google.

No news between 2023-11-15 00:00:00 and 2023-11-30 00:00:00 for Google.

No news between 2023-12-01 00:00:00 and 2023-12-15 00:00:00 for Google.

No news between 2023-12-15 00:00:00 and 2023-12-31 00:00:00 for Google.



In [6]:
# microsoft

company = "Microsoft"

for year in range(2018, 2023+1):

    headlines_df = get_headlines(year, company)

    file = "headlines/" + str(year) + "_" + company + "_headlines.csv"
    headlines_df.to_csv(file, index = False)

No news between 2023-07-15 00:00:00 and 2023-07-31 00:00:00 for Microsoft.

No news between 2023-08-01 00:00:00 and 2023-08-15 00:00:00 for Microsoft.

No news between 2023-08-15 00:00:00 and 2023-08-31 00:00:00 for Microsoft.

No news between 2023-09-01 00:00:00 and 2023-09-15 00:00:00 for Microsoft.

No news between 2023-09-15 00:00:00 and 2023-09-30 00:00:00 for Microsoft.

No news between 2023-10-01 00:00:00 and 2023-10-15 00:00:00 for Microsoft.

No news between 2023-10-15 00:00:00 and 2023-10-31 00:00:00 for Microsoft.

No news between 2023-11-01 00:00:00 and 2023-11-15 00:00:00 for Microsoft.

No news between 2023-11-15 00:00:00 and 2023-11-30 00:00:00 for Microsoft.

No news between 2023-12-01 00:00:00 and 2023-12-15 00:00:00 for Microsoft.

No news between 2023-12-15 00:00:00 and 2023-12-31 00:00:00 for Microsoft.



### 2.4 Combine data

In [81]:
import warnings
warnings.filterwarnings("ignore")

AAPL = pd.DataFrame()
MSFT = pd.DataFrame()
AMZN = pd.DataFrame()
GOOG = pd.DataFrame()
NVDA = pd.DataFrame()
for year in range(2018, 2023+1):
    AMZN = AMZN.append([pd.read_csv('headlines/' + str(year) + '_Amazon_headlines.csv')],
              ignore_index = True)
    AAPL = AAPL.append([pd.read_csv('headlines/' + str(year) + '_Apple_headlines.csv')],
              ignore_index = True)
    GOOG = GOOG.append([pd.read_csv('headlines/' + str(year) + '_Google_headlines.csv')],
              ignore_index = True)
    MSFT = MSFT.append([pd.read_csv('headlines/' + str(year) + '_Microsoft_headlines.csv')],
              ignore_index = True)
    NVDA = NVDA.append([pd.read_csv('headlines/' + str(year) + '_Nvidia_headlines.csv')],
              ignore_index = True)

In [82]:
AMZN.insert(1,'ticker', 'AMZN')
AAPL.insert(1, 'ticker', 'AAPL')
GOOG.insert(1, 'ticker', 'GOOG')
MSFT.insert(1, 'ticker', 'MSFT')
NVDA.insert(1, 'ticker', 'NVDA')

In [109]:
headlines = pd.concat([AMZN, AAPL, GOOG, MSFT, NVDA], ignore_index = True)
headlines['date'] = pd.to_datetime(headlines['date']).dt.tz_convert(None)
headlines.sort_values(by = ['date','ticker'], ignore_index = True, inplace = True)

In [110]:
headlines['publisher'] = headlines['publisher'].apply(lambda x: ' '.join(x.split()[3:]).strip("'").strip("}")[:-1])
headlines

,date,ticker,title,publisher
0,2018-01-01 08:00:00,AAPL,3 tips to maximize Apple's free Pages word pro...,TechRepublic
1,2018-01-01 08:00:00,AAPL,The 20 best iOS games of 2017 - Macworld,Macworld
2,2018-01-01 08:00:00,AAPL,How to Switch Apple Watch Home Screen from Gri...,Wccftech
3,2018-01-01 08:00:00,AMZN,"Will Ferrell, Molly Shannon Tease Trump and Ti...",Hollywood Reporter
4,2018-01-01 08:00:00,AMZN,The Limits of Amazon - WSJ - The Wall Street J...,The Wall Street Journal
...,...,...,...,...
65704,2023-07-11 19:49:00,GOOG,Google's Searchbot Could Put Me Out of a Job -...,The Atlantic
65705,2023-07-11 20:08:55,GOOG,Google On Fixing Discovered Currently Not Inde...,Search Engine Journal
65706,2023-07-11 20:16:08,GOOG,"Google’s head of AR software quits, citing “un...",Ars Technica
65707,2023-07-11 23:30:19,GOOG,Google quietly ditched plans for an AI-powered...,CNBC


In [115]:
headlines.to_csv('headlines.csv', index = False)

## 3. Add Sentiment Analysis to Headlines Dataset

In [45]:
headlines = pd.read_csv('headlines.csv')
headlines.head()

,date,ticker,title,publisher
0,2018-01-01 08:00:00,AAPL,3 tips to maximize Apple's free Pages word pro...,TechRepublic
1,2018-01-01 08:00:00,AAPL,The 20 best iOS games of 2017 - Macworld,Macworld
2,2018-01-01 08:00:00,AAPL,How to Switch Apple Watch Home Screen from Gri...,Wccftech
3,2018-01-01 08:00:00,AMZN,"Will Ferrell, Molly Shannon Tease Trump and Ti...",Hollywood Reporter
4,2018-01-01 08:00:00,AMZN,The Limits of Amazon - WSJ - The Wall Street J...,The Wall Street Journal


### 3.1 Cleaning

**Remove publisher tag -- this a rough solution that works for most tags**

In [48]:
def remove_source_tag(headline):
    """
    after each headline, there is a " - [source name]" like " - New York Times". 
    
    this function removes the final occurence of " - ..."
    it doesn't perfectly remove the source tags, ie " - WSJ - The Wall Street Journal" becomes "WSJ"
    
    headline: string
    """
    
    split_title = headline.split(' - ')
    n_splits = len(split_title) - 1
    
    return(' '.join(split_title[0:n_splits]))
    

In [51]:
def clean_headlines(df, start_date='2018-01-01', end_date='2023-07-01'):
    
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].apply(lambda ts: ts.replace(hour=0, minute=0, second=0))

    # date range
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    
    df = df.loc[(df['date'] >= start) & (df['date'] <= end)]
    
    # contains company names
    df = df.loc[pd.Series(
        df['title']
    ).str.contains("|".join(['Apple', 'Amazon', 'Google', 
                             'Microsoft','Nvidia']), case = False)]
    
    # remove duplicates
    df.drop_duplicates()
    
    df['title'] = df['title'].apply(lambda headline : remove_source_tag(headline))
    
    return df.reset_index(drop=True)

In [52]:
headlines = clean_headlines(headlines)

### 3.2 Add scores

In [53]:
def add_sentiment_scores(df):
    
    compound = []
    neg = []
    neu = []
    pos = []
    
    sid = SentimentIntensityAnalyzer()

    for headline in df['title']:
        ss = sid.polarity_scores(headline)

        compound.append(ss['compound'])
        neg.append(ss['neg'])
        neu.append(ss['neu'])
        pos.append(ss['pos'])
        
    df['compound'] = compound
    df['neg'] = neg
    df['neu'] = neu
    df['pos'] = pos
    
    return df

In [67]:
def wrangle_day(df):
    
    
    # number of headlines per day
    volume = df.groupby(['date', 'ticker']).count().iloc[:, 1]
    
    # take avg
    df = df.groupby(['date', 'ticker']).mean(numeric_only = True)
    df['volume'] = volume
    
    return df.reset_index().sort_values('date')
    

In [59]:
headlines = add_sentiment_scores(headlines)

In [56]:
headlines

,date,ticker,title,publisher,compound,neg,neu,pos
0,2018-01-01,AAPL,3 tips to maximize Apple's free Pages word pro...,TechRepublic,0.5106,0.000,0.708,0.292
1,2018-01-01,AAPL,How to Switch Apple Watch Home Screen from Gri...,Wccftech,0.0000,0.000,1.000,0.000
2,2018-01-01,AMZN,"Will Ferrell, Molly Shannon Tease Trump and Ti...",Hollywood Reporter,-0.3182,0.141,0.859,0.000
3,2018-01-01,AMZN,The Limits of Amazon WSJ,The Wall Street Journal,0.1779,0.000,0.702,0.298
4,2018-01-01,AMZN,Amazon Mistakenly Sends AWS Budget Emails Fore...,BleepingComputer,0.4215,0.167,0.455,0.379
...,...,...,...,...,...,...,...,...
56959,2023-07-01,AAPL,"New Apple Watch Ultra, 30-inch iMac make us sa...",Cult of Mac,0.0000,0.000,1.000,0.000
56960,2023-07-01,MSFT,"A quick look back at Microsoft Bob, which was ...",Neowin,-0.6249,0.215,0.785,0.000
56961,2023-07-01,AAPL,The Best Apple AirPods Models to Buy in 2023,IGN,0.6369,0.000,0.656,0.344
56962,2023-07-01,AAPL,How Google Beat Apple To AR And Still Failed,SlashGear,-0.5106,0.292,0.708,0.000


In [68]:
day_headlines = wrangle_day(headlines)
day_headlines

,date,ticker,compound,neg,neu,pos,volume
0,2018-01-01,AAPL,0.255300,0.000000,0.854000,0.146000,2
1,2018-01-01,AMZN,0.070300,0.077000,0.754000,0.169250,4
2,2018-01-01,GOOG,0.249680,0.000000,0.843000,0.157000,5
3,2018-01-01,MSFT,0.484700,0.081500,0.594500,0.324000,2
4,2018-01-02,AAPL,0.064093,0.093143,0.815571,0.091286,14
...,...,...,...,...,...,...,...
9263,2023-07-01,GOOG,-0.061820,0.071200,0.888800,0.040000,5
9264,2023-07-01,MSFT,-0.129689,0.067333,0.932667,0.000000,9
9261,2023-07-01,AAPL,0.112262,0.022462,0.897308,0.080231,13
9262,2023-07-01,AMZN,0.253340,0.040400,0.700600,0.258900,10


In [69]:
day_headlines.to_csv('daily_sentiment.csv', index=False)